In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2011','era_2011.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2011-01-01 ... 2011-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:57 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2011-01-01 ... 2011-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:57 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2011-01-01 00:00:00 1000.0         49.25    -124.85    73.575851  0.002427
                                            -124.60    62.771164  0.002087
                                            -124.35    51.093430  0.001852
                                            -124.10    45.622726  0.001813
                                            -123.85    49.282883  0.001925
...                                                          ...       ...
2011-12-31 21:00:00 1000.0         24.50    -68.10     86.628311  0.014817
                                            -67.85     86.155655  0.014671
                                            -67.60     84.931046  0.014435
                                            -67.35     83.214249  0.014140
                                            -67.10     81.188858  0.013852

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2011-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:57 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2011-12-31 1000.0         49.25    -124.85    84.160683  0.005769
                                   -124.60    81.390724  0.005762
                                   -124.35    78.797226  0.005697
                                   -124.10    76.269592  0.005647
                                   -123.85    73.418678  0.005660
...                                                 ...       ...
                          24.50    -68.10     77.497314  0.014261
                                   -67.85     77.515320  0.014251
                                   -67.60     77.536827  0.014243
                                   -67.35     77.523628  0.014230
                                   -67.10     77.469475  0.014208

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2011-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 84.16 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.005769 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:57 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2011_county_shapefile','tl_2011_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,72,045,01804502,72045,Comerío,Comerío Municipio,13,H1,G4020,490,41980,None,A,73557132,319735,+18.2250402,-066.2202984,"POLYGON ((-66.25713 18.24744, -66.25695 18.247..."
1,72,009,01804484,72009,Aibonito,Aibonito Municipio,13,H1,G4020,490,41980,None,A,81094444,71897,+18.1306472,-066.2639743,"POLYGON ((-66.25753 18.07588, -66.25937 18.077..."
2,72,039,01804499,72039,Ciales,Ciales Municipio,13,H1,G4020,490,41980,None,A,172309964,459615,+18.2912286,-066.5130169,"POLYGON ((-66.55771 18.27506, -66.55782 18.275..."
3,72,003,01804481,72003,Aguada,Aguada Municipio,13,H1,G4020,None,10380,None,A,79904253,38043835,+18.3756809,-067.1837074,"POLYGON ((-67.16355 18.41748, -67.16190 18.416..."
4,72,151,01804556,72151,Yabucoa,Yabucoa Municipio,13,H1,G4020,490,41980,None,A,143005203,72593008,+18.0598615,-065.8598555,"POLYGON ((-65.87885 18.02066, -65.87926 18.020..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,42,099,01213682,42099,Perry,Perry County,06,H1,G4020,276,25420,None,A,1428280078,10597982,+40.3977795,-077.2663284,"POLYGON ((-77.02920 40.38688, -77.02918 40.386..."
3230,40,061,01101818,40061,Haskell,Haskell County,06,H1,G4020,None,None,None,A,1493228577,126266671,+35.2322937,-095.1095776,"POLYGON ((-95.22136 35.27282, -95.22103 35.273..."
3231,17,197,01785190,17197,Will,Will County,06,H1,G4020,176,16980,16974,A,2167580642,31904479,+41.4484742,-087.9784564,"POLYGON ((-88.25535 41.55009, -88.25539 41.550..."
3232,40,081,01101828,40081,Lincoln,Lincoln County,06,H1,G4020,416,36420,None,A,2466481584,34614223,+35.7031175,-096.8813917,"POLYGON ((-97.14113 35.60848, -97.14119 35.609..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-66.25713 18.24744, -66.25695 18.247...
1       POLYGON ((-66.25753 18.07588, -66.25937 18.077...
2       POLYGON ((-66.55771 18.27506, -66.55782 18.275...
3       POLYGON ((-67.16355 18.41748, -67.16190 18.416...
4       POLYGON ((-65.87885 18.02066, -65.87926 18.020...
                              ...                        
3229    POLYGON ((-77.02920 40.38688, -77.02918 40.386...
3230    POLYGON ((-95.22136 35.27282, -95.22103 35.273...
3231    POLYGON ((-88.25535 41.55009, -88.25539 41.550...
3232    POLYGON ((-97.14113 35.60848, -97.14119 35.609...
3233    POLYGON ((-81.39113 41.46705, -81.39113 41.467...
Name: geometry, Length: 3234, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0    -66.257125  18.247439
     1    -66.256955  18.247451
     2    -66.256175  18.247371
     3    -66.255699  18.247497
     4    -66.255298  18.247541
...              ...        ...
3233 1161 -81.391230  41.463234
     1162 -81.391229  41.463273
     1163 -81.391213  41.463906
     1164 -81.391135  41.466968
     1165 -81.391134  41.467051

[7394987 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3234): ## A total of 3234 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 6min 23s
Wall time: 6min 30s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,<NA>,<NA>
1,<NA>,<NA>
2,<NA>,<NA>
3,<NA>,<NA>
4,<NA>,<NA>
...,...,...
3229,70.002977,0.007284
3230,60.415868,0.008541
3231,70.614322,0.007026
3232,53.137555,0.007493


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
GEOID         object
NAME          object
NAMELSAD      object
LSAD          object
CLASSFP       object
MTFCC         object
CSAFP         object
CBSAFP        object
METDIVFP      object
FUNCSTAT      object
ALAND          int64
AWATER         int64
INTPTLAT      object
INTPTLON      object
geometry    geometry
fips           int32
dtype: object

In [15]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','GEOID','NAMELSAD','LSAD','CLASSFP','MTFCC'
                             ,'CSAFP','CBSAFP','METDIVFP','FUNCSTAT','ALAND','AWATER','INTPTLAT',
                             'INTPTLON'], axis=1)
county_gdf

,NAME,geometry,fips
0,Comerío,"POLYGON ((-66.25713 18.24744, -66.25695 18.247...",72045
1,Aibonito,"POLYGON ((-66.25753 18.07588, -66.25937 18.077...",72009
2,Ciales,"POLYGON ((-66.55771 18.27506, -66.55782 18.275...",72039
3,Aguada,"POLYGON ((-67.16355 18.41748, -67.16190 18.416...",72003
4,Yabucoa,"POLYGON ((-65.87885 18.02066, -65.87926 18.020...",72151
...,...,...,...
3229,Perry,"POLYGON ((-77.02920 40.38688, -77.02918 40.386...",42099
3230,Haskell,"POLYGON ((-95.22136 35.27282, -95.22103 35.273...",40061
3231,Will,"POLYGON ((-88.25535 41.55009, -88.25539 41.550...",17197
3232,Lincoln,"POLYGON ((-97.14113 35.60848, -97.14119 35.609...",40081


In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Comerío,"POLYGON ((-66.25713 18.24744, -66.25695 18.247...",72045,<NA>,<NA>
1,Aibonito,"POLYGON ((-66.25753 18.07588, -66.25937 18.077...",72009,<NA>,<NA>
2,Ciales,"POLYGON ((-66.55771 18.27506, -66.55782 18.275...",72039,<NA>,<NA>
3,Aguada,"POLYGON ((-67.16355 18.41748, -67.16190 18.416...",72003,<NA>,<NA>
4,Yabucoa,"POLYGON ((-65.87885 18.02066, -65.87926 18.020...",72151,<NA>,<NA>
...,...,...,...,...,...
3229,Perry,"POLYGON ((-77.02920 40.38688, -77.02918 40.386...",42099,70.002977,0.007284
3230,Haskell,"POLYGON ((-95.22136 35.27282, -95.22103 35.273...",40061,60.415868,0.008541
3231,Will,"POLYGON ((-88.25535 41.55009, -88.25539 41.550...",17197,70.614322,0.007026
3232,Lincoln,"POLYGON ((-97.14113 35.60848, -97.14119 35.609...",40081,53.137555,0.007493


In [17]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
7,Cheshire,"POLYGON ((-72.20364 42.71954, -72.21044 42.719...",33005,72.483483,0.006312
8,Sheridan,"POLYGON ((-100.67346 47.58890, -100.67353 47.5...",38083,73.594511,0.005325
9,Morton,"POLYGON ((-101.28271 46.98107, -101.28226 46.9...",38059,72.187743,0.005373
10,Logan,"POLYGON ((-99.16207 46.63076, -99.16207 46.630...",38047,72.920394,0.00547
11,Eddy,"POLYGON ((-98.75588 47.58757, -98.75665 47.587...",38027,73.794728,0.005334
...,...,...,...,...,...
3229,Perry,"POLYGON ((-77.02920 40.38688, -77.02918 40.386...",42099,70.002977,0.007284
3230,Haskell,"POLYGON ((-95.22136 35.27282, -95.22103 35.273...",40061,60.415868,0.008541
3231,Will,"POLYGON ((-88.25535 41.55009, -88.25539 41.550...",17197,70.614322,0.007026
3232,Lincoln,"POLYGON ((-97.14113 35.60848, -97.14119 35.609...",40081,53.137555,0.007493


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
7,Cheshire,33005,72.483482622095948,0.006312352647880
8,Sheridan,38083,73.594510897944261,0.005325012338395
9,Morton,38059,72.187742993953648,0.005373025423763
10,Logan,38047,72.920393781093495,0.005469549026555
11,Eddy,38027,73.794727731382451,0.005334468130747
...,...,...,...,...
3229,Perry,42099,70.002977138613105,0.007283685198440
3230,Haskell,40061,60.415868398917837,0.008541429407694
3231,Will,17197,70.614322371874820,0.007025792856894
3232,Lincoln,40081,53.137554876723080,0.007493163289137


In [20]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
1494,Autauga,1001,66.510684759422816,0.009662284029450
2005,Baldwin,1003,67.253190657424369,0.010529259774927
127,Barbour,1005,64.834990422698880,0.009672891490234
2183,Bibb,1007,67.899477154363908,0.009640419810383
1069,Blount,1009,69.029146617201860,0.009445268368553
...,...,...,...,...
2650,Sweetwater,56037,60.459430166814137,0.003805057188664
2668,Teton,56039,65.982903561231979,0.003705465310060
2142,Uinta,56041,61.592614468288630,0.003595407907741
2152,Washakie,56043,52.780726918820385,0.003830232349566


In [21]:
county_var.to_pickle('era_2011.pkl')